# COALESCE,CONCAT
### 練習項目
- COALESCEによりnull値を埋める（※読み方は「コーアレス」らしい）
- CONCATにより文字列を結合する（BigQueryの場合は3文字以上指定可）

### 参考文献
***
- [SQL第2版ゼロからはじめるデータベース操作](https://www.shoeisha.co.jp/book/detail/9784798144450)
- [ビッグデータ分析・活用のためのSQLレシピ](https://book.mynavi.jp/supportsite/detail/9784839961268.html)

In [3]:
%reload_ext google.cloud.bigquery

In [4]:
import pandas as pd

## テーブル定義

In [27]:
p_id = 'tactile-acrobat-231812'
name = 'example.purchacse_log_with_coupon'
data = [
    (10001, 3280, None, '東京都', '千代田区'),
    (10002, 4650, 500, '東京都', '渋谷区'),
    (10003, 3870, None, '千葉県', '八王子'),
]
cols = ['purchase_id', 'amount', 'coupon', 'pref_name', 'city_name']

### CREATE

In [28]:
df_create = pd.DataFrame(data, columns=cols)
df_create.to_gbq(name, p_id)

1it [00:05,  5.11s/it]


### UPDATE

In [29]:
%%bigquery
UPDATE `tactile-acrobat-231812.example.purchacse_log_with_coupon` 
    set amount=100
WHERE
    dt = 'A'

""


### INSERT

In [42]:
%%bigquery
INSERT INTO `tactile-acrobat-231812.example.purchacse_log_with_coupon` 
    (dt, amount, agg_amount, year_avg_amount)
VALUES
    ('A',0,1,2),
    ('B',1,2,3)

""


### DELETE

In [39]:
%%bigquery
DELETE FROM `tactile-acrobat-231812.example.purchacse_log_with_coupon`
WHERE
    dt = '2015-01'

""


### DROP TABLE

In [26]:
%%bigquery
DROP TABLE `tactile-acrobat-231812.example.purchacse_log_with_coupon`

""


### SELECT（*）

In [29]:
%%bigquery df_ast
SELECT
  *
FROM 
  `tactile-acrobat-231812.example.purchacse_log_with_coupon`

In [30]:
display(df_ast)

,purchase_id,amount,coupon,pref_name,city_name
0,10003,3870,NaN,千葉県,八王子
1,10001,3280,NaN,東京都,千代田区
2,10002,4650,500.0,東京都,渋谷区


### COALESCEによりnull値を埋める & CONCATにより文字列を結合する

In [41]:
%%bigquery df_not_null_concat
SELECT
    purchase_id,
    amount,
    coupon,
    amount - coupon_not_null AS discount_amount,
    pref_city
FROM
    (
    SELECT
        purchase_id,
        amount,
        coupon,
        COALESCE(coupon, 0) AS coupon_not_null,
        CONCAT(pref_name, city_name) AS pref_city
    FROM 
      `tactile-acrobat-231812.example.purchacse_log_with_coupon`
    ) AS t

In [42]:
display(df_not_null_concat)

,purchase_id,amount,coupon,discount_amount,pref_city
0,10003,3870,NaN,3870.0,千葉県八王子
1,10001,3280,NaN,3280.0,東京都千代田区
2,10002,4650,500.0,4150.0,東京都渋谷区
